# Train classification

Most sharing code train this dataset as a regression task.

But in this code, I train as a classification task.

I encode the target value pressure to 950 classes and calculate CrossEntropy Loss.

In [1]:
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>")) 

In [2]:
# from kaggle_secrets import UserSecretsClient
# secret_label = "wandb"
# secret_value = UserSecretsClient().get_secret(secret_label)
# # !wandb login $secret_value

In [3]:
import gc
import os
import sys
import copy
import random
# import wandb
import math
from pathlib import Path
import scipy.signal
from scipy import signal

import numpy as np
import pandas as pd

from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import AdamW
from transformers import get_cosine_schedule_with_warmup
from sklearn.preprocessing import RobustScaler

device = torch.device("cuda")

In [4]:
sys.path.append('../../src/')
import utils as utils
from utils import Timer

In [5]:
class CFG:
    exp_num = 53
    n_folds = 5
    folds = [0, 1, 2, 3, 4]
    seed = 777
    local = True
    
    lr = 0.0015
    epochs = 100
    emb_dim = 64
    hidden_dim = 256
    weight_decay = 0.1    
    
    ######################
    # Loaders #
    ######################
    loader_params = {
        "train": {
            'batch_size': 128,
            'shuffle': True,
            'num_workers': 8,
            'pin_memory': True,
            'drop_last': True,
        },
        "valid": {
            'batch_size': 32,
            'shuffle': False,
            'num_workers': 8,
            'pin_memory': True,
            'drop_last': False,
        },
        "test": {
            'batch_size': 32,
            'shuffle': False,
            'num_workers': 8,
            'pin_memory': True,
            'drop_last': False,
        }
    }


In [6]:
utils.set_seed(CFG.seed)    

In [7]:
if CFG.local:
    DATA_DIR = Path("/home/knikaido/work/Ventilator-Pressure-Prediction/data/ventilator-pressure-prediction")
    OUTPUT_DIR = Path('./output/')
else:
    DATA_DIR = Path("../input/ventilator-pressure-prediction")
    OUTPUT_DIR = Path('')   

In [8]:
def loss_fn(y_pred, y_true, u_outs):
#     loss = nn.L1Loss()(y_pred.reshape(-1), y_true.reshape(-1))
    w = 1 - u_outs.reshape(-1)
    loss = nn.CrossEntropyLoss(reduction = 'none')(y_pred.reshape(-1, 950), y_true.reshape(-1)).reshape(-1)
    loss = loss * w
    loss = loss.sum() / w.sum()
    return loss

def compute_metric(preds, trues, u_outs):
    """
    Metric for the problem, as I understood it.
    """
    
    y = trues
    w = 1 - u_outs
    
    assert y.shape == preds.shape and w.shape == y.shape, (y.shape, preds.shape, w.shape)
    
    mae = w * np.abs(y - preds)
    mae = mae.sum() / w.sum()
    
    return mae

In [9]:
class VentilatorDataset(Dataset):
    
    def __init__(self, df, train_value_col, train_category_col, label_dic=None):
        self.dfs = [_df for _, _df in df.groupby("breath_id")]
        self.label_dic = label_dic
        
    def __len__(self):
        return len(self.dfs)
    
    def __getitem__(self, item):
        df = self.dfs[item]
        X = df[train_category_col + train_value_col].values
        u_out = df['u_out'].values
        y = df['pressure'].values
        if self.label_dic is None:
            label = [-1]
        else:
            label = [self.label_dic[i] for i in y]

        d = {
            "X": torch.tensor(X).float(),
            "u_out": torch.tensor(u_out).long(),
            "y" : torch.tensor(label).long(),
        }
        return d

In [10]:
class VentilatorModel(nn.Module):
    
    def __init__(self, input_dim):
        super(VentilatorModel, self).__init__()
        self.rc_emb = nn.Embedding(9, 4, padding_idx=0)

        self.seq_emb = nn.Sequential(
            nn.Linear(4+input_dim, CFG.emb_dim),
            nn.LayerNorm(CFG.emb_dim)
        )
        
        self.lstm = nn.LSTM(CFG.emb_dim, CFG.hidden_dim, batch_first=True, bidirectional=True, dropout=0.1, num_layers=4)

        self.head = nn.Sequential(
            nn.Linear(CFG.hidden_dim * 2, CFG.hidden_dim * 2),
            nn.LayerNorm(CFG.hidden_dim * 2),
            nn.ReLU(),
            nn.Linear(CFG.hidden_dim * 2, 950),
        )
        
        # Encoder
        initrange = 0.1
        self.rc_emb.weight.data.uniform_(-initrange, initrange)
        
        # LSTM
        for n, m in self.named_modules():
            if isinstance(m, nn.LSTM):
                print(f'init {m}')
                for param in m.parameters():
                    if len(param.shape) >= 2:
                        nn.init.orthogonal_(param.data)
                    else:
                        nn.init.normal_(param.data)

    def forward(self, X, y=None):
        # embed
        bs = X.shape[0]
        rc_emb = self.rc_emb(X[:,:,0].long()).view(bs, 80, -1)

        seq_x = torch.cat((rc_emb, X[:, :, 1:]), 2)
        emb_x = self.seq_emb(seq_x)
        
        out, _ = self.lstm(emb_x, None) 
        logits = self.head(out)

        return logits

In [11]:
def train_loop(model, optimizer, scheduler, loader):
    losses, lrs = [], []
    model.train()
    optimizer.zero_grad()
    for d in loader:
        out = model(d['X'].to(device))
        loss = loss_fn(out, d['y'].to(device), d['u_out'].to(device))
        losses.append(loss.item())
        step_lr = np.array([param_group["lr"] for param_group in optimizer.param_groups]).mean()
        lrs.append(step_lr)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        scheduler.step()

    return np.array(losses).mean(), np.array(lrs).mean()


def valid_loop(model, loader, target_dic_inv):
    losses, predicts = [], []
    model.eval()
    for d in loader:
        with torch.no_grad():
            out = model(d['X'].to(device))
            loss = loss_fn(out, d['y'].to(device), d['u_out'].to(device))
        pred = out.reshape(-1, 950).softmax(1)
        pred = torch.sum(torch.tensor(unique_targets).to(device) *  pred, axis=1)
        losses.append(loss.item())
        predicts.append(pred.cpu().numpy())

    return np.array(losses).mean(), np.concatenate(predicts)

def test_loop(model, loader, target_dic_inv):
    predicts = []
    model.eval()
    for d in loader:
        with torch.no_grad():
            out = model(d['X'].to(device))
        pred = out.reshape(-1, 950).softmax(1)
        pred = torch.sum(torch.tensor(unique_targets).to(device) *  pred, axis=1)
        predicts.append(pred.cpu().numpy())

    return np.concatenate(predicts)

In [12]:
def get_raw_features(input_df, dataType = 'train'):
    colum = ['time_step', 'u_in', 'u_out']

    return input_df[colum]

In [13]:
def get_category_features(input_df, dataType = 'train'):
    output_df = copy.deepcopy(input_df)
    colum = ['R_C']
    rc_map = {'5_10': 0, '5_20': 1, '5_50': 2, '20_10': 3, '20_20': 4, '20_50': 5, '50_10': 6, '50_20': 7, '50_50': 8}
    
    output_df['R_C'] = [f'{r}_{c}' for r, c in zip(output_df['R'], output_df['C'])]
    output_df['R_C'] = output_df['R_C'].map(rc_map)

    return output_df[colum]

In [14]:
def get_simple_calc_features(input_df, dataType = 'train'):
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    output_df['time_delta'] = output_df.groupby('breath_id')['time_step'].diff().fillna(0)
    output_df['delta'] = output_df['time_delta'] * output_df['u_in']
    output_df['area'] = output_df.groupby('breath_id')['delta'].cumsum()

    output_df['cross']= output_df['u_in']*output_df['u_out']
    output_df['cross2']= output_df['time_step']*output_df['u_out']
    
    output_df['u_in_cumsum'] = (output_df['u_in']).groupby(output_df['breath_id']).cumsum()
    output_df['one'] = 1
    output_df['count'] = (output_df['one']).groupby(output_df['breath_id']).cumsum()
    output_df['u_in_cummean'] =output_df['u_in_cumsum'] / output_df['count']
    
    output_df = output_df.drop(['count','one'], axis=1)
    
    return output_df.iloc[:, c_num:]

In [15]:
def get_diff_shift_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    USE_LAG = [-2, -1, 1, 2, 3, 4]
    
    for lag in USE_LAG:
        output_df[f'breath_id_lag{lag}']=output_df['breath_id'].shift(lag).fillna(0)
        output_df[f'breath_id_lag{lag}same']=np.select([output_df[f'breath_id_lag{lag}']==output_df['breath_id']], [1], 0)

        # u_in 
        output_df[f'u_in_lag_{lag}'] = output_df['u_in'].shift(lag).fillna(0) * output_df[f'breath_id_lag{lag}same']
        output_df[f'u_in_diff_{lag}'] = output_df['u_in'] - output_df[f'u_in_lag_{lag}']
        output_df[f'u_out_lag_{lag}'] = output_df['u_out'].shift(lag).fillna(0) * output_df[f'breath_id_lag{lag}same']

        # breath_time
    output_df[f'time_step_lag_{1}'] = output_df['time_step'].shift(1).fillna(0) * output_df[f'breath_id_lag{1}same']
    output_df[f'time_step_diff_{1}'] = output_df['time_step'] - output_df[f'time_step_lag_{1}']

    drop_columns = ['time_step_lag_1']
    drop_columns += [f'breath_id_lag{i}' for i in USE_LAG]
    drop_columns += [f'breath_id_lag{i}same' for i in USE_LAG]
    output_df = output_df.drop(drop_columns, axis=1)

    # fill na by zero
    output_df = output_df.fillna(0)
    
    return output_df.iloc[:, c_num:]

In [16]:
def get_agg_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    # Dict for aggregations
    create_feature_dict = {
        'u_in': [np.max, np.mean],
    }
    
    def get_agg_window(start_time=0, end_time=3.0, add_suffix = False):
        
        df_tgt = output_df[(output_df['time_step'] >= start_time) & (output_df['time_step'] <= end_time)]
        df_feature = df_tgt.groupby(['breath_id']).agg(create_feature_dict)
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(start_time) + '_' + str(end_time))
            
        return df_feature
    
    df_agg_feature = get_agg_window().reset_index()
    
#     df_tmp = get_agg_window(start_time = 2, add_suffix = True).reset_index()
#     df_agg_feature = df_agg_feature.merge(df_tmp, how = 'left', on = 'breath_id')
#     df_tmp = get_agg_window(start_time = 1, add_suffix = True).reset_index()
#     df_agg_feature = df_agg_feature.merge(df_tmp, how = 'left', on = 'breath_id')
#     df_tmp = get_agg_window(end_time = 1, add_suffix = True).reset_index()
#     df_agg_feature = df_agg_feature.merge(df_tmp, how = 'left', on = 'breath_id')
#     df_tmp = get_agg_window(end_time = 2, add_suffix = True).reset_index()
#     df_agg_feature = df_agg_feature.merge(df_tmp, how = 'left', on = 'breath_id')

    output_df = pd.merge(output_df, df_agg_feature, how='left', on='breath_id')
    
    output_df['u_in_diffmax'] = output_df['u_in_amax'] - output_df['u_in']
    output_df['u_in_diffmean'] = output_df['u_in_mean'] - output_df['u_in']
    
#     output_df = output_df.drop(['u_in_amax','u_in_mean'], axis=1)
    
    return output_df.iloc[:, c_num:]

In [17]:
def lowpass(x, samplerate, fp, fs, gpass, gstop):
    fn = samplerate / 2   #ナイキスト周波数
    wp = fp / fn  #ナイキスト周波数で通過域端周波数を正規化
    ws = fs / fn  #ナイキスト周波数で阻止域端周波数を正規化
    N, Wn = signal.buttord(wp, ws, gpass, gstop)  #オーダーとバターワースの正規化周波数を計算
    b, a = signal.butter(N, Wn, "low")            #フィルタ伝達関数の分子と分母を計算
    y = signal.filtfilt(b, a, x)                  #信号に対してフィルタをかける
    return y  

In [18]:
fp = 5 # 通過域端周波数[Hz]
fs = 10 # 阻止域端周波数[Hz]
gpass = 3 # 通過域端最大損失[dB]
gstop = 40 # 阻止域端最小損失[dB]
samplerate = 100
def lowpass_filter(series):
    return lowpass(series, samplerate, fp, fs, gpass, gstop)

In [19]:
def get_filter_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    def get_agg_window(start_time=0, end_time=3.0, add_suffix = False):
        
        df_tgt = output_df[(output_df['time_step'] >= start_time) & (output_df['time_step'] <= end_time)]
        df_feature = df_tgt.groupby(['breath_id'])['u_in'].apply(lowpass_filter)
        df_feature.name = 'u_in_filter'
                    
        return df_feature
    
    df_agg_feature = get_agg_window().reset_index()
    df_agg_feature = df_agg_feature.explode("u_in_filter").reset_index(drop=True)
    
    USE_LAG = [-2, -1, 1, 2, 3, 4]
    
    for lag in USE_LAG:
        df_agg_feature[f'breath_id_lag{lag}']=df_agg_feature['breath_id'].shift(lag).fillna(0)
        df_agg_feature[f'breath_id_lag{lag}same']=np.select([df_agg_feature[f'breath_id_lag{lag}']==df_agg_feature['breath_id']], [1], 0)

        # u_in 
        df_agg_feature[f'u_in_filter_lag_{lag}'] = df_agg_feature['u_in_filter'].shift(lag).fillna(0) * df_agg_feature[f'breath_id_lag{lag}same']
        df_agg_feature[f'u_in_filter_diff_{lag}'] = df_agg_feature['u_in_filter'] - df_agg_feature[f'u_in_filter_lag_{lag}']

    drop_columns = [f'breath_id_lag{i}' for i in USE_LAG]
    drop_columns += [f'breath_id_lag{i}same' for i in USE_LAG]
    df_agg_feature = df_agg_feature.drop(drop_columns, axis=1)
    df_agg_feature = df_agg_feature.fillna(0)
    
    return df_agg_feature.iloc[:, 1:]

In [20]:
def to_feature(input_df, dataType = 'train'):
    """input_df を特徴量行列に変換した新しいデータフレームを返す.
    """

    processors = [
        get_raw_features,
        get_simple_calc_features,
        get_diff_shift_features,
        get_agg_features,
        get_category_features,
        get_filter_features
    ]

    out_df = pd.DataFrame()

    for func in tqdm(processors, total=len(processors)):
        with Timer(prefix='' + func.__name__ + ' '):
            _df = func(input_df, dataType)

        # 長さが等しいことをチェック (ずれている場合, func の実装がおかしい)
        assert len(_df) == len(input_df), func.__name__
        out_df = pd.concat([out_df, _df], axis=1)
#     out_df = utils.reduce_mem_usage(out_df)
    
    return out_df

In [21]:
train = pd.read_csv(DATA_DIR / 'train.csv')
test = pd.read_csv(DATA_DIR / 'test.csv')
sub_df = pd.read_csv(DATA_DIR / "sample_submission.csv")

In [22]:
display(train), display(test)

,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,1,20,50,0.000000,0.083334,0,5.837492
1,2,1,20,50,0.033652,18.383041,0,5.907794
2,3,1,20,50,0.067514,22.509278,0,7.876254
3,4,1,20,50,0.101542,22.808822,0,11.742872
4,5,1,20,50,0.135756,25.355850,0,12.234987
...,...,...,...,...,...,...,...,...
6035995,6035996,125749,50,10,2.504603,1.489714,1,3.869032
6035996,6035997,125749,50,10,2.537961,1.488497,1,3.869032
6035997,6035998,125749,50,10,2.571408,1.558978,1,3.798729
6035998,6035999,125749,50,10,2.604744,1.272663,1,4.079938


,id,breath_id,R,C,time_step,u_in,u_out
0,1,0,5,20,0.000000,0.000000,0
1,2,0,5,20,0.031904,7.515046,0
2,3,0,5,20,0.063827,14.651675,0
3,4,0,5,20,0.095751,21.230610,0
4,5,0,5,20,0.127644,26.320956,0
...,...,...,...,...,...,...,...
4023995,4023996,125748,20,10,2.530117,4.971245,1
4023996,4023997,125748,20,10,2.563853,4.975709,1
4023997,4023998,125748,20,10,2.597475,4.979468,1
4023998,4023999,125748,20,10,2.631134,4.982648,1


(None, None)

In [23]:
train_df = to_feature(train, dataType = 'train')
test_df = to_feature(test, dataType = 'test')

  0%|          | 0/6 [00:00<?, ?it/s]

get_raw_features  0.023[s]
get_simple_calc_features  12.133[s]
get_diff_shift_features  2.150[s]
get_agg_features  1.039[s]
get_category_features  2.391[s]
get_filter_features  41.357[s]


  0%|          | 0/6 [00:00<?, ?it/s]

get_raw_features  0.013[s]
get_simple_calc_features  7.864[s]
get_diff_shift_features  1.234[s]
get_agg_features  0.615[s]
get_category_features  1.503[s]
get_filter_features  27.127[s]


In [24]:
train_category_col = ['R_C']
train_value_col = [i for i in train_df.columns.to_list() if i not in train_category_col]

In [25]:
norm_features = train_value_col
norm_features = sorted(list(set(train_value_col) - set(['u_out'])), key=norm_features.index)
def norm_scale(train_df, test_df):
    scaler = RobustScaler()
    all_u_in = np.vstack([train_df[norm_features].values, test_df[norm_features].values])
    scaler.fit(all_u_in)
    train_df[norm_features] = scaler.transform(train_df[norm_features].values)
    test_df[norm_features] = scaler.transform(test_df[norm_features].values)
    return train_df, test_df

In [ ]:
train_df, test_df = norm_scale(train_df, test_df)

In [ ]:
train_df = utils.reduce_mem_usage(train_df)
test_df = utils.reduce_mem_usage(test_df)

In [ ]:
train_df = pd.concat([train_df, train[['id', 'breath_id', 'pressure']]], axis=1)
test_df = pd.concat([test_df, test[['id', 'breath_id']]], axis=1)
test_df['pressure'] = train_df['pressure'].values[-1]

In [ ]:
unique_targets = sorted(train_df['pressure'].unique().tolist())
target_dic = {v:i for i, v in enumerate(sorted(train_df['pressure'].unique().tolist()))}
target_dic_inv = {v: k for k, v in target_dic.items()}

In [ ]:
display(train_df), display(test_df)

In [ ]:
oof = np.zeros(len(train_df))
test_preds_lst = []
input_dim = len(train_value_col)
train_df['pred'] = 0

gkf = GroupKFold(n_splits=CFG.n_folds).split(train_df, train_df.pressure, groups=train_df.breath_id)
for fold, (_, valid_idx) in enumerate(gkf):
    train_df.loc[valid_idx, 'fold'] = fold
    
for i, fold in enumerate(range(CFG.n_folds)):
    if i not in CFG.folds:
        continue
    print(f'Fold-{fold}')
    
    trn_df = train_df.query(f"fold!={fold}").reset_index(drop=True)
    val_df = train_df.query(f"fold=={fold}").reset_index(drop=True)
    
    loaders = {
        phase: DataLoader(
            VentilatorDataset(
                df_, train_value_col, train_category_col, target_dic
            ),
            **CFG.loader_params[phase])  # type: ignore
        for phase, df_ in zip(["train", "valid", "test"], [trn_df, val_df, test_df])
    }
    

    model = VentilatorModel(input_dim)
    model.to(device)

    optimizer = AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
    num_train_steps = int(len(loaders['train']) * CFG.epochs)
    num_warmup_steps = int(num_train_steps / 10)
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps)

    model_path = OUTPUT_DIR / f"ventilator_f{fold}_best_model.bin"

    valid_best_score = float('inf')
    valid_best_score_mask = float('inf')
    for epoch in tqdm(range(CFG.epochs)):
        train_loss, lrs = train_loop(model, optimizer, scheduler, loaders['train'])
        valid_loss, valid_predict = valid_loop(model, loaders['valid'], target_dic_inv)
       
        valid_score_mask = compute_metric(valid_predict, val_df['pressure'].values, val_df['u_out'].values)

        print(f"epoch = {epoch}, valid mask score = {valid_score_mask}:")

        if valid_score_mask < valid_best_score_mask:
            valid_best_score_mask = valid_score_mask
            torch.save(model.state_dict(), model_path)
            oof[train_df.query(f"fold=={fold}").index.values] = valid_predict

        torch.cuda.empty_cache()
        gc.collect()

    model.load_state_dict(torch.load(model_path))
    test_preds = test_loop(model, loaders['test'], target_dic_inv)
    test_preds_lst.append(test_preds)
    
    sub_df['pressure'] = test_preds
    sub_df.to_csv(OUTPUT_DIR / f"sub_f{fold}.csv", index=None)
    
    valid_loss, valid_predict = valid_loop(model, loaders['valid'], target_dic_inv)
    valid_score_mask = compute_metric(valid_predict, val_df['pressure'].values, val_df['u_out'].values)
    print(f"fold = {epoch}, valid mask score = {valid_score_mask}:")
    train_df.loc[train_df['fold'] == fold, 'pred'] = valid_predict
    train_df.loc[train_df['fold'] == fold, ['id', 'pred']].to_csv(OUTPUT_DIR / f"oof_{fold}.csv", index=None)

    torch.cuda.empty_cache()
    gc.collect()

In [36]:
valid_score_mask = compute_metric(train_df['pred'].values, train_df['pressure'].values, train_df['u_out'].values)
print("CV:", valid_score_mask)

CV: 0.1640938902946353


In [ ]:
oof_df = train_df.loc[:, ['id', 'pred']]
oof_df.to_csv(OUTPUT_DIR / "oof_total.csv", index=None)

In [ ]:
sub_df['pressure'] = np.mean(np.stack(test_preds_lst), axis=0)
sub_df.to_csv(OUTPUT_DIR / "submission_mean.csv", index=None)

sub_df['pressure'] = np.median(np.stack(test_preds_lst), axis=0)
sub_df.to_csv(OUTPUT_DIR / "submission_median.csv", index=None)

# Post Processing: https://www.kaggle.com/snnclsr/a-dummy-approach-to-improve-your-score-postprocess
unique_pressures = train_df["pressure"].unique()
sorted_pressures = np.sort(unique_pressures)
total_pressures_len = len(sorted_pressures)

def find_nearest(prediction):
    insert_idx = np.searchsorted(sorted_pressures, prediction)
    if insert_idx == total_pressures_len:
        # If the predicted value is bigger than the highest pressure in the train dataset,
        # return the max value.
        return sorted_pressures[-1]
    elif insert_idx == 0:
        # Same control but for the lower bound.
        return sorted_pressures[0]
    lower_val = sorted_pressures[insert_idx - 1]
    upper_val = sorted_pressures[insert_idx]
    return lower_val if abs(lower_val - prediction) < abs(upper_val - prediction) else upper_val

sub_df = pd.read_csv(OUTPUT_DIR / "submission_mean.csv")
sub_df["pressure"] = sub_df["pressure"].apply(find_nearest)
sub_df.to_csv(OUTPUT_DIR / "submission_mean_pp.csv", index=None)

sub_df = pd.read_csv(OUTPUT_DIR / "submission_median.csv")
sub_df["pressure"] = sub_df["pressure"].apply(find_nearest)
sub_df.to_csv(OUTPUT_DIR / "submission_median_pp.csv", index=None)